In [1]:
# LIBRERIAS
import pandas as pd
import numpy as np
import math as math
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

Haremos el ejemplo de esta aplicación para una observación, cojeremos la observación número 3

In [2]:
df = pd.read_csv("df_modelo.csv")
X_y1 = pd.read_csv("X_y1.csv")
X_y2 = pd.read_csv("X_y2.csv")
y1 = df["booking_score"]
y2 = df["review_score"]

In [3]:
del df["Unnamed: 0"]
del X_y1["Unnamed: 0"]
del X_y2["Unnamed: 0"]

In [4]:
id = 3

X_y1_test = X_y1.loc[X_y1.index == id,:]
X_y2_test = X_y2.loc[X_y1.index == id,:]
X_y1_train = X_y1.drop(id)
X_y2_train = X_y2.drop(id)
y1_train = y1.drop(id)
y2_train = y2.drop(id)
y1_test = y1[y1.index == id]
y2_test = y2[y2.index == id]

Parametros para el modelo de y1:

{'objective':'reg:linear',
'max_depth': 7,
'learning_rate': 0.005,
'colsample_bytree': 0.45,
'n_estimators' = 2100}

Parametros para el modelo de y2:

{'objective':'reg:linear', 
'max_depth': 10, 
'learning_rate': 0.01, 
'colsample_bytree': 0.45, 
'min_child_weight': 10, 
'min_split_loss': 10,
'n_estimators' = 450}

Variables que se pueden modificar en X_y1: "price", "minimum_nights", "Shower gel", "host_response_time" y "license".
Variables que se pueden modificar en X_y2: "host_acceptance_rate", "minimum_nights", "Coffe maker", "Dining table", "Shower gel", "Lock on bedroom door", "Cleaning products", "Wine glasses", "reviews", "host_respose_time", "host_is_superhost", "license" y "instant_bookable".

In [5]:
data_dmatrix_y1 = xgb.DMatrix(data=X_y1,label=y1)

xg_reg_y1 = xgb.XGBRegressor(objective = 'reg:linear',max_depth =  7, learning_rate =  0.005, colsample_bytree = 0.45,  n_estimators = 2100)

xg_reg_y1.fit(X_y1_train,y1_train)
pred_y1 = xg_reg_y1.predict(X_y1_test)
pred_train_y1 = xg_reg_y1.predict(X_y1_train)

[17:06:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [6]:
data_dmatrix_y2 = xgb.DMatrix(data=X_y2,label=y2)

xg_reg_y2 = xgb.XGBRegressor(objective = 'reg:linear', max_depth = 10, learning_rate = 0.01, colsample_bytree = 0.45, min_child_weight = 10, min_split_loss = 10,  n_estimators = 450)

xg_reg_y2.fit(X_y2_train,y2_train)
pred_y2 = xg_reg_y2.predict(X_y2_test)
pred_train_y2 = xg_reg_y2.predict(X_y2_train)

[17:06:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [7]:
scores_y1 = np.append(pred_train_y1,pred_y1)
scores_y2 = np.append(pred_train_y2,pred_y2)
scores_y1 = list(scores_y1)
scores_y2 = list(scores_y2)
scores_y1 = [5 if ele > 5 else ele for ele in scores_y1]
scores_y1 = [0 if ele < 0 else ele for ele in scores_y1]
scores_y2 = [5 if ele > 5 else ele for ele in scores_y2]
scores_y2 = [0 if ele < 0 else ele for ele in scores_y2]
mixed_score = [x + y for x, y in zip(scores_y1, scores_y2)]
score_pred = pred_y1 + pred_y2
mixed_score.sort(reverse=True)
(mixed_score.index(score_pred)/len(df)) * 100

31.51977060066405

In [8]:
# FUNCION PARA VARIABLES NUMERICAS
def set_numeric_variable_value(series, variable):
    message = "The value of " + variable + " is " + str(series[variable].values[0]) + " . Set new value (enter to remain the same): "
    new_value_str = input(message)
    if (not new_value_str == ""):
        series[variable] = float(new_value_str)
        
# FUNCION PARA VARIABLES CATEGORICAS
def set_dummy_variable_value(series, variable, dictionary):
    cont = 0
    index = 0
    for i in range(0,len(dictionary[variable])-1):
        if (dictionary[variable][i] in list(series.columns)):
            if((series[dictionary[variable][i]] == 1).bool()):
                val = dictionary[variable][i]
                cont = 1
        else:
            index = i
    if (cont == 0):
         val = dictionary[variable][index]
    
    ins = input("The value of " + variable + " is " + val + " : Select which one to set to 1: " + str([(i,dictionary[variable][i]) for i in range(len(dictionary[variable]))]) + "(enter to remain the same)")
    
    if (ins != ""):
        for value in dictionary[variable]:
            if (value in list(series.columns)):
                series[value] = 0
        if((series[dictionary[variable][int(ins)]].name in list(series.columns))):
            series[dictionary[variable][int(ins)]] = 1

# FUNCION PARA TRANSFORMAR EL TEST
def test_change(series1, series2, variables, dictionary):
    for name in variables:
        if (name in list(dictionary.keys())):
            if (name in list(series1.columns)):
                set_dummy_variable_value(series1, name, dictionary)
                if (name in list(series2.columns)):
                    series2[name] = series1[name]
            else:
                set_dummy_variable_value(series2, name, dictionary)
        else:
            if (name in list(series1.columns)):
                set_numeric_variable_value(series1, name)
                if (name in list(series2.columns)):
                    series2[name] = series1[name]
            else:
                set_numeric_variable_value(series2,name)

# FUNCION PARA CALCULAR EL RANKING
def ranking_score(X_y1_train, X_y2_train, X_y1_test, X_y2_test, xg_reg1, xg_reg2):
    pred_train_y1 = xg_reg1.predict(X_y1_train)
    pred_train_y2 = xg_reg2.predict(X_y2_train)
    pred_test_y1 = xg_reg1.predict(X_y1_test)
    pred_test_y2 = xg_reg2.predict(X_y2_test)
    scores_y1 = np.append(pred_train_y1,pred_test_y1)
    scores_y2 = np.append(pred_train_y2,pred_test_y2)
    scores_y1 = list(scores_y1)
    scores_y2 = list(scores_y2)
    scores_y1 = [5 if ele > 5 else ele for ele in scores_y1]
    scores_y1 = [0 if ele < 0 else ele for ele in scores_y1]
    scores_y2 = [5 if ele > 5 else ele for ele in scores_y2]
    scores_y2 = [0 if ele < 0 else ele for ele in scores_y2]
    mixed_score = [x + y for x, y in zip(scores_y1, scores_y2)]
    score_pred = pred_test_y1 + pred_test_y2
    mixed_score.sort(reverse=True)
    rank = (mixed_score.index(score_pred)/len(df)) * 100
    return rank
    

In [9]:
#set_dummy_variable_value(X_y1_test, 'license')
dictionary = {"license" : ["license_Exempt", "license_Unknown", "license_Yes"],
              "host_response_time" : ["host_response_time_Unknown", "host_response_time_a few days or more","host_response_time_within a day", 
                                      "host_response_time_within a few hours", "host_response_time_within an hour"],
              "instant_bookable" : ["instant_bookable_f", "instant_bookable_t"],
              "host_is_superhost" : ["host_is_superhost_f", "host_is_superhost_t"]}

list1 = ["price", "minimum_nights", "Shower gel", "Crib", "host_response_time", "license"]
list2 = ["host_acceptance_rate", "minimum_nights", "Coffee maker", "Shower gel", "Crib", "Wifi", "Cleaning products", 
        "Wine glasses", "reviews", "host_response_time", "host_is_superhost", "license", "instant_bookable"]

list_t = list1 + list2
var = list(np.unique(list_t))


In [11]:
test_change(X_y1_test, X_y2_test, var, dictionary)

C:\Users\Andreu\AppData\Local\Temp\ipykernel_968\1745960534.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series[variable] = float(new_value_str)
C:\Users\Andreu\AppData\Local\Temp\ipykernel_968\1745960534.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series2[name] = series1[name]


In [12]:
ranking_score(X_y1_train, X_y2_train, X_y1_test, X_y2_test, xg_reg_y1, xg_reg_y2)

20.057349833987324